In [1]:
from functools import reduce

In [2]:
def hex2bin(data):
    bits = bin(int(data, 16))[2:]
    return ''.join(['0'] * (len(data) * 4 - len(bits))) + bits

def bin2int(data):
    return int(data, 2)

In [3]:
LITTERAL_TYPE = 4

class Packet():
    def __init__(self, bits):
        if len(bits) < 6:
            raise ValueError('')
        
        self.version = bin2int(bits[:3])
        self.type = bin2int(bits[3:6])
        self.consumed_bits = 6
        self.inner_packets = []
        
        if self.type == LITTERAL_TYPE:
            self.litteral = ''
            while True:
                self.litteral += bits[self.consumed_bits+1: self.consumed_bits+5]
                self.consumed_bits += 5
                if bits[self.consumed_bits - 5] == '0':
                    break
            self.litteral = bin2int(self.litteral)
        else:
            if bits[self.consumed_bits] == '0':
                self.consumed_bits += 1
                self.inner_packets_length = bin2int(bits[self.consumed_bits:self.consumed_bits+15])
                self.consumed_bits += 15
                inner_bits = bits[self.consumed_bits:self.consumed_bits+self.inner_packets_length]
                while True:
                    try:
                        packet = Packet(inner_bits)
                    except:
                        break
                    self.inner_packets.append(packet)
                    inner_bits = inner_bits[packet.consumed_bits:]
                    self.consumed_bits += packet.consumed_bits
            else:
                self.consumed_bits += 1
                self.n_inner_packets = bin2int(bits[self.consumed_bits:self.consumed_bits+11])
                self.consumed_bits += 11
                inner_bits = bits[self.consumed_bits:]
                for i in range(self.n_inner_packets):
                    packet = Packet(inner_bits)
                    self.inner_packets.append(packet)
                    inner_bits = inner_bits[packet.consumed_bits:]
                    self.consumed_bits += packet.consumed_bits
    
    def __repr__(self, depth=0):
        s = ''.join(['-'] * depth)
        s += f'Packet [version={self.version}, type={self.type}'
        if hasattr(self, 'litteral'):
            s += f', litteral={self.litteral}'
            
        if hasattr(self, 'inner_packets_length'):
            s += f', inner_packets_length={self.inner_packets_length}'
        
        if hasattr(self, 'n_inner_packets'):
            s += f', n_inner_packets={self.n_inner_packets}'
        
        s += ']'
        if self.inner_packets:
            s += f' with {len(self.inner_packets)} children :'
            for p in self.inner_packets:
                s +='\n' + p.__repr__(depth=depth+1)
        return s

In [4]:
def sum_versions(packet):
    return packet.version + sum(map(lambda p: sum_versions(p), packet.inner_packets))

def expression_value(packet):
    if hasattr(packet, 'litteral'):
        return packet.litteral
    
    inner_values = list(map(lambda p: expression_value(p), packet.inner_packets))
    if packet.type == 0:
        return sum(inner_values)
    elif packet.type == 1:
        return reduce(lambda x, y: x * y, inner_values, 1)
    elif packet.type == 2:
        return min(inner_values)
    elif packet.type == 3:
        return max(inner_values)
    elif packet.type == 5:
        return int(inner_values[0] > inner_values[1])
    elif packet.type == 6:
        return int(inner_values[0] < inner_values[1])
    elif packet.type == 7:
        return int(inner_values[0] == inner_values[1])
    else:
        print('error:')

In [5]:
with open('input.txt', 'r') as f:
    data = f.readline().rstrip()

In [6]:
bits = hex2bin(data)
top_packet = Packet(bits)
top_packet

Packet [version=2, type=0, n_inner_packets=53] with 53 children :
-Packet [version=3, type=2, n_inner_packets=2] with 2 children :
--Packet [version=0, type=4, litteral=31]
--Packet [version=6, type=4, litteral=180539]
-Packet [version=7, type=1, n_inner_packets=2] with 2 children :
--Packet [version=0, type=5, n_inner_packets=2] with 2 children :
---Packet [version=5, type=4, litteral=56]
---Packet [version=3, type=4, litteral=2844922]
--Packet [version=6, type=4, litteral=110]
-Packet [version=2, type=1, n_inner_packets=2] with 2 children :
--Packet [version=4, type=7, inner_packets_length=57] with 2 children :
---Packet [version=4, type=4, litteral=30749]
---Packet [version=4, type=4, litteral=334951]
--Packet [version=6, type=4, litteral=2388881186]
-Packet [version=5, type=1, n_inner_packets=3] with 3 children :
--Packet [version=5, type=4, litteral=123]
--Packet [version=2, type=4, litteral=160]
--Packet [version=4, type=4, litteral=44]
-Packet [version=5, type=0, inner_packets_l

In [7]:
print(f'Part 1: {sum_versions(top_packet)}')

Part 1: 993


In [8]:
print(f'Part 2: {expression_value(top_packet)}')

Part 2: 144595909277
